In [1]:
import instructor
from pydantic import BaseModel, Field
from openai import AzureOpenAI
from enum import Enum
from typing import List
from dotenv import load_dotenv
import os,json
import pandas as pd

In [2]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [3]:
load_dotenv("C:\\Users\\sourabh.mohite\\personal_repo\\SQLAnalytics\\Q&A-and-RAG-with-SQL-and-TabularData\\.env")

True

In [4]:
# --------------------------------------------------------------
# Regular Completion using OpenAI (with drawbacks)
# --------------------------------------------------------------

client = AzureOpenAI(
    api_version=os.getenv("OPENAI_API_VERSION"),
    azure_endpoint=os.getenv("OPENAI_API_BASE"),
    api_key=os.getenv("OPENAI_API_KEY"),
    azure_deployment=os.getenv("GET_DEPLOYMENT_NAME")
)

In [5]:
conversation = """{'OP':'Can I know what problem you are facing with your mobile device?', 
'CU':'My screen got damaged, it has cracks all over it.', 
'OP':'I’m sorry to hear that. How did the damage occur?', 
'CU':'I dropped it accidentally while walking.', 
'OP':'Thank you for the information. Is the device still under warranty?', 
'CU':'Yes, I purchased it just 6 months ago, so it should be covered.', 
'OP':'Great! I will guide you through the claim process. You will need to send the device for an inspection. Can I have your contact details to start the process?', 
'CU':'Sure, my name is John Doe, and my phone number is 123-456-7890.', 
'OP':'Thank you, John. I’ve noted your details. You’ll receive an email with further instructions on sending the device for inspection. Is there anything else I can assist you with today?', 
'CU':'No, that will be all for now. Thank you for your help!', 
'OP':'You’re welcome! We’ll be in touch soon. Have a great day!'}"""

In [6]:
# --------------------------------------------------------------
# Step 2: Patch your LLM with instructor
# --------------------------------------------------------------

# Instructor makes it easy to get structured data like JSON from LLMs
client = instructor.patch(client)

# --------------------------------------------------------------
# Step 3: Define Pydantic data models
# --------------------------------------------------------------

"""
This code defines a structured data model for classifying customer support tickets using Pydantic and Python's Enum class. 
It specifies categories, urgency levels, customer sentiments, and other relevant information as predefined options or constrained fields. 
This structure ensures data consistency, enables automatic validation, and facilitates easy integration with AI models and other parts of a support ticket system.
"""

class CallCategory(str, Enum):
    ORDER_ISSUE = "order_issue"
    ACCOUNT_ACCESS = "account_access"
    PRODUCT_INQUIRY = "product_inquiry"
    TECHNICAL_SUPPORT = "technical_support"
    BILLING = "billing"
    DEVICEDAMAGE = "device_damage"
    OTHER = "other"
    

class CustomerSentiment(str, Enum):
    ANGRY = "angry"
    FRUSTRATED = "frustrated"
    NEUTRAL = "neutral"
    SATISFIED = "satisfied"

class AgentSentiment(str, Enum):
    ANGRY = "angry"
    FRUSTRATED = "frustrated"
    NEUTRAL = "neutral"

class AgentBehavior(str, Enum):
    HELPFUL = "helpful"
    PATIENT = "patient"
    IMPATIENT = "impatient"
    AGGRESSIVE = "aggressive"
    NEUTRAL = "neutral"
    CONFIDENT = "confident"
    APOLOGETIC = "apologetic"
    PROFESSIONAL = "professional"

class AgentFaul(str, Enum):
    FAUL = "faul"
    NOFAUL = "no-faul"

class CustomerEsclation(str, Enum):
    ESCALATION = "esclation"
    NOESCALATION = "no-escalation"

class CallUrgency(str, Enum):
    LOW = "low"
    MEDIUM = "medium"
    HIGH = "high"
    CRITICAL = "critical"

class CallClassification(BaseModel):
    category: CallCategory
    urgency: CallUrgency
    customer_sentiment: CustomerSentiment
    agent_behavior:AgentBehavior
    agent_faul : AgentFaul
    customer_escalation : CustomerEsclation
    agent_rating: float = Field(ge=0, le=5, description="Based on conversation give rating to agent")
    confidence: float = Field(ge=0, le=1, description="Confidence score for the classification")
    device_infromation: List[str] = Field(description="Based on conversation extract device information(like device details, model, device type etc.)")
    key_information: List[str] = Field(description="Give 7-8 linear breif summary of the call")
    suggested_action: str = Field(description="Brief suggestion for handling the ticket")
 

In [7]:
SYSTEM_PROMPT = """ You are an AI assistant for a claim management system at a Insurance Company.
Your role is to analyze incoming claim calls related to mobile devices and provide structured information to help our team respond quickly and effectively.

Business Context:

We handle thousands of claims daily, including warranty claims, damage claims, technical issues, and billing disputes.
Effective categorization, prioritization, and accurate responses are key to maintaining high customer satisfaction and operational efficiency.
Claims are processed based on urgency, warranty status, agent behaviour and customer sentiment.
Your Tasks:

Categorize the claim into the most appropriate category (e.g., device damage, warranty issue, technical issue, billing dispute).
Assess the urgency of the issue (low, medium, high, critical).
Determine the Customer's sentiment based on the tone of the message (positive, neutral, negative).
Determine the Agent's sentiment based on the tone of the message (positive, neutral, negative).
Determine the agent's behaviour based on the tone (helpful,patient).
Determine the agent's used faul language or not (faul or no-faul).
Determine the agent's rating between 1-5 (e.g 1-poor and 5-excellent).
Determine the wheather there is any escalation happend (escalation ,no-escalation).
7-8 line Breif Summary of the call that would be helpful for the claim processing team.
Based on conversation extract device information (like device details, model, device type etc.)
Suggest an initial action for processing the claim (e.g., inspect the device, escalate to technician, check warranty status).
Provide a confidence score for your classification based on the provided information.
Instructions:

Be objective and base your analysis solely on the information provided in the claim call conversation.
If unsure about any aspect, reflect that in your confidence score.
For key_information, extract specific details like device model, damage type, purchase date, serial number, or any additional facts.
The suggested_action should be clear and actionable to facilitate quick resolution.
Remember to consider the customer sentiment and adjust your urgency assessment accordingly.
"""

In [8]:
conversation = """{
"CU": "I’ve been waiting on hold for 45 minutes! This is absolutely ridiculous. What’s going on with my claim?"
"OP": "Well, it’s not like you’re the only one with a claim. We’re dealing with a lot of cases, and your impatience isn’t helping."
"CU": "Impatience? I just want my issue resolved! I’ve been promised a resolution, and nothing has happened yet!"
"OP": "Look, your claim is in the queue, alright? You’ll get your payout when it’s processed, so relax. It’s not like it’s the end of the world."
"CU": "You’re seriously telling me to relax? You’ve had my claim for weeks, and I’ve been given no updates!"
"OP": "Well, we don’t give updates every minute. Maybe if you were more patient, you’d get the information when it’s ready."
"CU": "I don’t care about your excuses. I expect a call back by the end of the day or I’ll escalate this."
"OP": "You can escalate it if you want, but it’s not going to speed things up. We’ll get to it when we get to it."
"CU": "I’ve had enough of this nonsense. Get me your supervisor, now!"
"OP": "Fine, I’ll transfer you. But don’t expect them to say anything different. Good luck."
"CU": "Unbelievable. Just get me the supervisor!"
}"""

In [9]:
conversation = """{
  "OP": "おはようございます。故障紛失サポートセンター岩津でございます。",
  "CU": "すみませんどうも。",
  "OP": "はい。",
  "CU": "タブレットなんですけど。",
  "OP": "そうすると、ええ。",
  "CU": "あの電源が入らなくなったんで。",
  "OP": "はい。電源は7はい。",
  "CU": "あー入れ仕事で使うもんで。あのもう何何とか電源がはい。",
  "OP": "はい。",
  "CU": "今すぐしたいんですけども。",
  "OP": "はい。えっと電源入らなくなったのは今がはじめてでございますか。",
  "CU": "次のもうあの始まっとったもんで電話機の時の僕9時ちょっと前ぐらいに。",
  "OP": "ええ。",
  "CU": "はい。",
  "OP": "あのーあれしちゃうよな再起動したんですよね。",
  "CU": "再起動。はい。",
  "OP": "再起動したらもうそれっきり電源が入らなくて。",
  "CU": "はい。",
  "OP": "なるほど。",
  "CU": "これあのー充電しても充電の5時からでますっていうのはあの。",
  "OP": "はい。",
  "CU": "すぐに。",
  "OP": "と申します。今はあのすぐにお客様そこ外からお電話いただいてる状態ですか。あーすみません。あて先なんですね。えっとーそしたらじゃって電源がつなげない状態ですよね。",
  "CU": "ま。",
  "OP": "そうですね02ですね。",
  "CU": "はい。",
  "OP": "おーそれでま充電器繋いでも充電されない状態な。",
  "CU": "はい。",
  "OP": "はい。",
  "CU": "わかりました。そしたらですねちょっとその今、まずすぐ試せる方法としては強制再起動という方があるんですが、ちょっとその。",
  "CU": "あーそれも機能を見て全部調べてやったんですけどね。",
  "OP": "強制再起動をえっと今ちなみにお使いの機種っておわかりますか。",
  "CU": "えーとこれは電話なえーっとXperiaの、あー。",
  "OP": "要はタブレットはいいの。",
  "CU": "XperiaのSOT31側に使っていうことだと思うんです。2020ね。1本です。",
  "OP": "はい。",
  "CU": "あー。",
  "OP": "ご担当その音量を上げる方のキーを同時に3秒以上の長押しが強制再起動となるんですけども。これですか。",
  "CU": "これの番組だめだけどもSIMカード入ってるよこの赤いボタンを。",
  "OP": "はい。",
  "CU": "もうこれもしたんですけども。",
  "OP": "うん。",
  "CU": "俺も全然だめで。",
  "OP": "はい。",
  "CU": "うーんそうですか。",
  "OP": "えーと直ちにメモリーカードとかは入ってますか。",
  "CU": "SD。",
  "OP": "あメモリーカードはじゃないです。",
  "CU": "は入っていない。",
  "OP": "そうですねまたあの入ってるだけにどうぞもしG1回新田でして。",
  "CU": "すみません。あとか。",
  "OP": "抜き差しした、あー。",
  "CU": "不具合なく2回ばかりやったんですけども。その辺でだめでね。",
  "OP": "はい。",
  "CU": "はい。",
  "OP": "そのまだ使ったね。",
  "CU": "うーん。",
  "OP": "でもう。",
  "CU": "うん。",
  "OP": "今まで何もそういう不具合のままで。",
  "CU": "ええ。",
  "OP": "うん。",
  "CU": "うーんそうですね。もう時々にこーお電話いただくなんかね、急に電源が入らなくなったんだってお客様に対しは結構電源は強制再起動っていうものがかなり有効だったりするんで、それでんー繋がる方っていうの結構多いんですね。",
  "CU": "はい。",
  "OP": "うん。",
  "CU": "はいはい。",
  "OP": "でそれをちょっとし、。",
  "CU": "情報でも昨日あの同意ででauのあの再度か何か事情やり方出たもんで全部あのしたんだね。",
  "OP": "本当ですか。はい。",
  "CU": "そうですか。強制再起動も全くもう試しいただいても、。",
  "OP": "あー。",
  "CU": "5分と思った。",
  "OP": "5004画面ね。",
  "CU": "うん。",
  "OP": "そうすると、。",
  "CU": "何か、。",
  "OP": "ちなみにどこしたこともない自然にはしたこともないし。",
  "CU": "ええ。",
  "OP": "なるほど。はい。",
  "CU": "504のとこにずっと置きっ放しってことで今はもうし。",
  "OP": "うん。はー。",
  "CU": "はい。",
  "OP": "で数も3年ぐらいしか経ってないね。",
  "CU": "はい。うーん直接、。",
  "OP": "70やとか早すぎるもんでえー。",
  "CU": "まいたしますん例えば、。",
  "OP": "まもし電池の劣化とかがあったとしてでもまその場合は、もうま電池もちがだんだん短くなってきてるかとか。",
  "CU": "うん。",
  "OP": "うん。",
  "CU": "電源落ちの症状が出てくるようになったとかも何かしら洗浄あったりするんですけどそういったものもなかった状態ですかね。",
  "CU": "いやま電池の減りが早いかっていうのはもういろんなもんでその再起動する段階で見た50％。",
  "OP": "50％はいじゃもう。",
  "CU": "うん。",
  "OP": "それに電池劣化たとしても多くの再起動とか強制再起動では立ち上がってくると思うんですけれどもっていう、。",
  "CU": "はい。",
  "OP": "うん。",
  "CU": "うーんそうか。",
  "OP": "後はえっとまそしたらですねもう本当切り分けとして考えてくしかないんですけども。",
  "CU": "うん。",
  "OP": "えっとちょっと別の充電器を差し込んだ時の充電ランプ。",
  "CU": "はい。",
  "OP": "うー何とかつかどうかっていうことになってくるんですが。",
  "CU": "上のあの届いかね。あの何でかっていうようなやつAdobeまとまってなかったもんですからもんで。",
  "OP": "はい。",
  "CU": "はい。",
  "OP": "はい。",
  "CU": "上今あるそのー全部でいい何本かその充電器があるんですけどそれやったらどれ多少です。",
  "OP": "全部だめですか。",
  "CU": "うん。",
  "OP": "まおその他のやつ他のことには使えるんで。あのそれではないとね。",
  "CU": "うーん充電器側の問題ではないということなんですね。",
  "OP": "多分充電器があのちょっとあのーあ電話のやつでやったわけではないもんで何ともいえないんですけども。どの車も何もあるやつではあとりあえずやっちゃうんだけど何ともない。",
  "CU": "はい。うーん。",
  "OP": "あー。",
  "CU": "うん。",
  "OP": "そうしたらそうですね。えっと今この私の方でもご案内できるものっていうのもすでにお客様の方でお試しいただいてた強制再起動ですとかを繰り返して電源が立ち上がらないという状態ですし、充電器を変えても充電できない。",
  "CU": "うん。",
  "OP": "それを踏まえてみますと本格的に点検修理が必要な状況かなと思われます。",
  "CU": "まこれからですね。",
  "OP": "はい。修理に関しましては当店にお送りいただくことになるのですが、万が一部品交換の必要があった場合、修理費用がかかる場合もございます。",
  "CU": "修理費用は別の。最初に言ってください。料金表ありますからいきますから。うん。",
  "OP": "はい。分かりました。先にその必要かかかり。",
  "CU": "じゃあ確認してみてください。"
}
"""

In [10]:
def classify_ticket(ticket_text: str) -> CallClassification:
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        response_model = CallClassification,
        temperature=0,
        max_retries=3,
        messages=[
            {
                "role": "system",
                "content": SYSTEM_PROMPT,
            },
            {"role": "user", "content": ticket_text}
        ]
    )
    return response

result = classify_ticket(conversation)
print(result.model_dump_json(indent=2))
df = pd.DataFrame([[conversation]], columns=['conversation'])
df1 = pd.DataFrame([json.loads(result.model_dump_json(indent=2))])
df2 = pd.concat([df, df1], axis=1)
display(df2)


{
  "category": "technical_support",
  "urgency": "high",
  "customer_sentiment": "neutral",
  "agent_behavior": "helpful",
  "agent_faul": "no-faul",
  "customer_escalation": "no-escalation",
  "agent_rating": 4.0,
  "confidence": 0.8,
  "device_infromation": [
    "Xperia SOT31"
  ],
  "key_information": [
    "Customer's tablet is not turning on",
    "Customer has tried forced restart and changing chargers",
    "Customer has been experiencing occasional power issues",
    "Customer has not tried cleaning the charging port"
  ],
  "suggested_action": "Advise customer to send the tablet for inspection and repair"
}


,conversation,category,urgency,customer_sentiment,agent_behavior,agent_faul,customer_escalation,agent_rating,confidence,device_infromation,key_information,suggested_action
0,"{\n ""OP"": ""おはようございます。故障紛失サポートセンター岩津でございます。"",\n ""CU"": ""すみませんどうも。"",\n ""OP"": ""はい。"",\n ""CU"": ""タブレットなんですけど。"",\n ""OP"": ""そうすると、ええ。"",\n ""CU"": ""あの電源が入らなくなったんで。"",\n ""OP"": ""はい。電源は7はい。"",\n ""CU"": ""あー入れ仕事で使うもんで。あのもう何何とか電源がはい。"",\n ""OP"": ""はい。"",\n ""CU"": ""今すぐしたいんですけども。"",\n ""OP"": ""はい。えっと電源入らなくなったのは今がはじめてでございますか。"",\n ""CU"": ""次のもうあの始まっとったもんで電話機の時の僕9時ちょっと前ぐらいに。"",\n ""OP"": ""ええ。"",\n ""CU"": ""はい。"",\n ""OP"": ""あのーあれしちゃうよな再起動したんですよね。"",\n ""CU"": ""再起動。はい。"",\n ""OP"": ""再起動したらもうそれっきり電源が入らなくて。"",\n ""CU"": ""はい。"",\n ""OP"": ""なるほど。"",\n ""CU"": ""これあのー充電しても充電の5時からでますっていうのはあの。"",\n ""OP"": ""はい。"",\n ""CU"": ""すぐに。"",\n ""OP"": ""と申します。今はあのすぐにお客様そこ外からお電話いただいてる状態ですか。あーすみません。あて先なんですね。えっとーそしたらじゃって電源がつなげない状態ですよね。"",\n ""CU"": ""ま。"",\n ""OP"": ""そうですね02ですね。"",\n ""CU"": ""はい。"",\n ""OP"": ""おーそれでま充電器繋いでも充電されない状態な。"",\n ""CU"": ""はい。"",\n ""OP"": ""はい。"",\n ""CU"": ""わかりました。そしたらですねちょっとその今、まずすぐ試せる方法としては強制再起動という方があるんですが、ちょっとその。"",\n ""CU"": ""あーそれも機能を見て全部調べてやったんですけどね。"",\n ""OP"": ""強制再起動をえっと今ちなみにお使いの機種っておわかりますか。"",\n ""CU"": ""えーとこれは電話なえーっとXperiaの、あー。"",\n ""OP"": ""要はタブレットはいいの。"",\n ""CU"": ""XperiaのSOT31側に使っていうことだと思うんです。2020ね。1本です。"",\n ""OP"": ""はい。"",\n ""CU"": ""あー。"",\n ""OP"": ""ご担当その音量を上げる方のキーを同時に3秒以上の長押しが強制再起動となるんですけども。これですか。"",\n ""CU"": ""これの番組だめだけどもSIMカード入ってるよこの赤いボタンを。"",\n ""OP"": ""はい。"",\n ""CU"": ""もうこれもしたんですけども。"",\n ""OP"": ""うん。"",\n ""CU"": ""俺も全然だめで。"",\n ""OP"": ""はい。"",\n ""CU"": ""うーんそうですか。"",\n ""OP"": ""えーと直ちにメモリーカードとかは入ってますか。"",\n ""CU"": ""SD。"",\n ""OP"": ""あメモリーカードはじゃないです。"",\n ""CU"": ""は入っていない。"",\n ""OP"": ""そうですねまたあの入ってるだけにどうぞもしG1回新田でして。"",\n ""CU"": ""すみません。あとか。"",\n ""OP"": ""抜き差しした、あー。"",\n ""CU"": ""不具合なく2回ばかりやったんですけども。その辺でだめでね。"",\n ""OP"": ""はい。"",\n ""CU"": ""はい。"",\n ""OP"": ""そのまだ使ったね。"",\n ""CU"": ""うーん。"",\n ""OP"": ""でもう。"",\n ""CU"": ""うん。"",\n ""OP"": ""今まで何もそういう不具合のままで。"",\n ""CU"": ""ええ。"",\n ""OP"": ""うん。"",\n ""CU"": ""うーんそうですね。もう時々にこーお電話いただくなんかね、急に電源が入らなくなったんだってお客様に対しは結構電源は強制再起動っていうものがかなり有効だったりするんで、それでんー繋がる方っていうの結構多いんですね。"",\n ""CU"": ""はい。"",\n ""OP"": ""うん。"",\n ""CU"": ""はいはい。"",\n ""OP"": ""でそれをちょっとし、。"",\n ""CU"": ""情報でも昨日あの同意ででauのあの再度か何か事情やり方出たもんで全部あのしたんだね。"",\n ""OP"": ""本当ですか。はい。"",\n ""CU"": ""そうですか。強制再起動も全くもう試しいただいても、。"",\n ""OP"": ""あー。"",\n ""CU"": ""5分と思った。"",\n ""OP"": ""5004画面ね。"",\n ""CU"": ""うん。"",\n ""OP"": ""そうすると、。"",\n ""CU"": ""何か、。"",\n ""OP"": ""ちなみにどこしたこともない自然にはしたこともないし。"",\n ""CU"": ""ええ。"",\n ""OP"": ""なるほど。はい。"",\n ""CU"": ""504のとこにずっと置きっ放しってことで今はもうし。"",\n ""OP"": ""うん。はー。"",\n ""CU"": ""はい。"",\n ""OP"": ""で数も3年ぐらいしか経ってないね。"",\n ""CU"": ""はい。うーん直接、。"",\n ""OP"": ""70やとか早すぎるもんでえー。"",\n ""CU"": ""まいたしますん例えば、。"",\n ""OP"": ""まもし電池の劣化とかがあったとしてでもまその場合は、もうま電池もちがだんだん短くなってきてるかとか。"",\n ""CU"": ""うん。"",\n ""OP"": ""うん。"",\n ""CU"": ""電源落ちの症状が出てくるようになったとかも何かしら洗浄あったりするんですけどそういったものもなかった状態ですかね。"",\n ""CU"": ""いやま電池の減りが早いかっていうのはもういろんなもんでその再起動する段階で見た50％。"",\n ""OP"": ""50％はいじゃもう。"",\n ""CU"": ""うん。"",\n ""OP"": ""それに電池劣化たとしても多くの再起動とか強制再起動では立ち上がってくると思うんですけれどもっていう、。"",\n ""CU"": ""はい。"",\n ""OP"": ""うん。"",\n ""CU"": ""うーんそうか。"",\n ""OP"": ""後はえっとまそしたらですねもう本当切り分けとして考えてくしかないんですけども。"",\n ""CU"": ""うん。"",\n ""OP"": ""えっとちょっと別の充電器を差し込んだ時の充電ランプ。"",\n ""CU"": ""はい。"",\n ""OP"": ""うー何とかつかどうかっていうことになってくるんですが。"",\n ""CU"": ""上のあの届いかね。あの何でかっていうようなやつAdobeまとまってなかったもんですからもんで。"",\n ""OP"": ""はい。"",\n ""CU"": ""はい。"",\n ""OP"": ""はい。"",\n ""CU"": ""上今あるそのー全部でいい何本かその充電器があるんですけどそれやったらどれ多少です。"",\n ""OP"": ""全部だめですか。"",\n ""CU"": ""うん。"",\n ""OP"": ""まおその他のやつ他のことには使えるんで。あのそれではないとね。"",\n ""CU"": ""うーん充電器側の問題ではないということなんですね。"",\n ""OP"": ""多分充電器があのちょっとあのーあ電話のやつでやったわけではないもんで何ともいえないんですけども。どの車も何もあるやつではあとりあえずやっちゃうんだけど何ともない。"",\n ""CU"": ""はい。うーん。"",\n ""OP"": ""あー。"",\n ""CU"": ""うん。"",\n ""OP"": ""そうしたらそうですね。えっと今この私の方でもご案内できるものっていうのもすでにお客様の方でお試しいただいてた強制再起動ですとかを繰り返して電源が立ち上がらないという状態ですし、充電器を変えても充電できない。

In [11]:
from langchain_core.prompts import PromptTemplate
prompt = PromptTemplate.from_template(
    """{SYSTEM_PROMPT}"""
)

In [12]:
from langchain_experimental.llms.ollama_functions import OllamaFunctions
def classify_ticket(ticket_text: str) -> CallClassification:
    llm = OllamaFunctions(model="llama3", format="json", temperature=0)
    structured_llm = llm.with_structured_output(CallClassification)
    chain = prompt | structured_llm
    print(chain.invoke(f"{ticket_text}"))

result = classify_ticket(conversation)
print(result.model_dump_json(indent=2))

C:\Users\sourabh.mohite\AppData\Local\Temp\ipykernel_39828\55630568.py:3: LangChainDeprecationWarning: The class `OllamaFunctions` was deprecated in LangChain 0.0.64 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = OllamaFunctions(model="llama3", format="json", temperature=0)


ValueError: Failed to parse a response from llama3 output: { "OP": "Good morning. I'm from the Support Center for Faulty Devices in Japan." }

  







: 